# Support Vector Machine

In [4]:
%pip install -q pandas matplotlib numpy scikit-learn ace_tools panelsplit shap upgrade jupyter ipywidgets 
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install thundersvm-cu90-0.2.0-py3-none-linux_x86_64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'thundersvm==cu90': Expected end or semicolon (after name and no valid version specifier)
    thundersvm==cu90
              ^


In [5]:
from thundersvm import SVC 
import torch
import shap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import zipfile
import os

from panelsplit.cross_validation import PanelSplit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterSampler
from tqdm import tqdm

import random

FileNotFoundError: Please build the library first!

In [3]:
# Define file names
csv_filename = "../final_df.csv"
zip_filename = "../final_df.zip"

# Extract the CSV file from the ZIP
with zipfile.ZipFile(zip_filename, 'r') as zipf:
    zipf.extract(csv_filename)

# Read the CSV file into a DataFrame
final_df = pd.read_csv("final_df.csv")

# Delete the extracted CSV file after reading
os.remove("final_df.csv")

In [ ]:
#Getting rid of redundant columns
# X final
X1 = final_df.drop(columns=['year', 'codmpio', 'cluster_kmeans', 'Departamento', 'Municipio', 'Region', 'pc_loss', 'f_loss', 'areaoficialhm2', 'gdp']) # GDP, area already out since the were already considered
X1 = X1.drop(columns=['CV-01-15','CV-03-64', 'CV-03-26b', 'CV-01-11', 'CV-01-1', 'PCC-02-0', 'PCC-02-3', 'EIS-00-0', 'PCC-00-0', 'CTI-00-0', 'C-02-8t', 'ICM-00-0',
                      'EIS-03-4', 'CTI-01-3', 'SEG-00-0', 'SEG-01-6', 'SOS-00-0', 'SOS-02-0', 'SOS-02-2', 'SOS-01-6', 'GPI-00-0', 'GPI-02-4', 'GPI-02-5', 'P-01-34-1',
                      'P-01-46', 'P-01-25', 'CV-02-12e', 'CV-03-51', 'total_ac']) 

print(len(X1.columns))
# Keep feature names
original_feature_names = list(X1.columns) 

# y 
y = final_df['pc_loss']
y = np.log1p(final_df['pc_loss'])

# Normalize output
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)

71


In [ ]:
# Set seed for reproducibility
seed_value = 17
np.random.seed(seed_value)

# 🔹 Define SVR Model Parameters Grid
param_dist_svr = {
    "C": [0.1, 1, 10, 100],            # Regularization parameter
    "epsilon": [0.01, 0.1, 1],           # Epsilon in the epsilon-SVR model
    "kernel": ["rbf", "linear"],         # Kernel type
    "gamma": ["scale", "auto"]           # Kernel coefficient for 'rbf'
}

# 🔹 Define Time Series Split (PanelSplit for panel data)
panel_split = PanelSplit(periods=final_df.year, n_splits=5)

# 🔹 Store All Results for SVR
cv_results_svr = []
saved_models_svr = []

# Compute total iterations across all folds (for progress tracking)
total_iters = 100 * panel_split.n_splits  # Adjust if needed

# Prepare Randomized Parameter Search for SVR
param_list_svr = list(ParameterSampler(param_dist_svr, n_iter=100, random_state=seed_value))

# Global progress bar across parameter sets
with tqdm(total=len(param_list_svr), desc="🔍 Searching Hyperparameters (Cross-Validated)") as pbar:
    for params in param_list_svr:
        fold_metrics = []
        fold_models = []
        
        # Cross-validation using PanelSplit
        for fold, (train_idx, test_idx) in enumerate(panel_split.split(X1)):
            # Train-test split
            X_train, X_test = X1[train_idx], X1[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            # 🔹 Preprocess: Standardize features (essential for SVM)
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # 🔹 Define and train SVR model
            svr_model = SVR(**params)
            svr_model.fit(X_train_scaled, y_train)
            
            # 🔹 Predict
            y_test_pred = svr_model.predict(X_test_scaled)
            y_train_pred = svr_model.predict(X_train_scaled)
            
            # 🔹 Compute Metrics
            mse = mean_squared_error(y_test, y_test_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_test_pred)
            r2_test = r2_score(y_test, y_test_pred)
            r2_train = r2_score(y_train, y_train_pred)
            
            fold_metrics.append({
                "MSE": mse,
                "RMSE": rmse,
                "MAE": mae,
                "R2_test": r2_test,
                "R2_train": r2_train
            })
            fold_models.append(svr_model)
        
        # 🔹 Average Metrics across folds
        avg_metrics = {
            "MSE": np.mean([m["MSE"] for m in fold_metrics]),
            "RMSE": np.mean([m["RMSE"] for m in fold_metrics]),
            "MAE": np.mean([m["MAE"] for m in fold_metrics]),
            "R2_test": np.mean([m["R2_test"] for m in fold_metrics]),
            "R2_train": np.mean([m["R2_train"] for m in fold_metrics])
        }
        
        saved_models_svr.append({
            "params": params,
            "metrics": avg_metrics,
            "models": fold_models  # Optionally select the best model among folds
        })
        pbar.update(1)


c:\Users\cpedr\anaconda3\envs\nlp\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
🔍 Searching Hyperparameters (Cross-Validated):   2%|▏         | 1/48 [02:02<1:36:04, 122.65s/it]

In [ ]:
import numpy as np
import cupy as cp
from cuml.svm import SVR as cumlSVR
from cuml.preprocessing import StandardScaler as cumlStandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

# Set seed for reproducibility
seed_value = 17
np.random.seed(seed_value)

# Assuming final_df and PanelSplit are defined as in your original code:
# For example, PanelSplit may look like:
# panel_split = PanelSplit(periods=final_df.year, n_splits=5)
panel_split = PanelSplit(periods=final_df.year, n_splits=5)

# For demonstration, store metrics from each fold
cv_results_gpu = []

# Global progress bar for iterating folds (or you can wrap hyperparameter tuning later)
with tqdm(total=panel_split.n_splits, desc="Processing Folds on GPU") as pbar:
    for fold, (train_idx, test_idx) in enumerate(panel_split.split(X1)):
        # Split training and test sets using indices
        X_train, X_test = X1[train_idx], X1[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Convert data to CuPy arrays for GPU processing
        X_train_cp = cp.asarray(X_train)
        X_test_cp = cp.asarray(X_test)
        y_train_cp = cp.asarray(y_train)

        # Standardize features using cuML's StandardScaler
        scaler = cumlStandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_cp)
        X_test_scaled = scaler.transform(X_test_cp)

        # Define and train the GPU-enabled SVR model
        # (Here we use a basic kernel and hyperparameters; these can be tuned later)
        svr_gpu = cumlSVR(kernel="rbf", C=1.0, epsilon=0.1)
        svr_gpu.fit(X_train_scaled, y_train_cp)

        # Predict on training and test sets
        y_train_pred_cp = svr_gpu.predict(X_train_scaled)
        y_test_pred_cp = svr_gpu.predict(X_test_scaled)

        # Convert predictions from CuPy to NumPy for metric evaluation
        y_train_pred = cp.asnumpy(y_train_pred_cp)
        y_test_pred = cp.asnumpy(y_test_pred_cp)

        # Evaluate predictions using scikit-learn metrics
        mse = mean_squared_error(y_test, y_test_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_test_pred)
        r2_test = r2_score(y_test, y_test_pred)
        r2_train = r2_score(y_train, y_train_pred)

        cv_results_gpu.append({
            "fold": fold,
            "MSE": mse,
            "RMSE": rmse,
            "MAE": mae,
            "R2_test": r2_test,
            "R2_train": r2_train
        })
        
        pbar.update(1)

# Print or log the cross-validation results
print("GPU-accelerated SVR cross-validation results:")
for res in cv_results_gpu:
    print(res)
